## Step 0: Install docker and place sudo password in ~/.sudopassword

### Removed previous run containers

In [1]:
!sudo -S docker rm /zookeeper-server < ~/.sudopassword
!sudo -S docker rm /kafka-server < ~/.sudopassword
!sudo -S docker rm /event-stream < ~/.sudopassword
!sudo -S docker network rm app-tier < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream
[sudo] password for kalinga: app-tier


## Step 1: Create a network

In [2]:
!sudo -S docker network create app-tier --driver bridge < ~/.sudopassword

[sudo] password for kalinga: d143c8de739b6bd53a30792618e9d80c2b3c30ec1a049c36563bec43262bc239


## Step 2: Launch the Zookeeper server instance

In [3]:
!sudo -S  docker run -d --name zookeeper-server --network app-tier -e ALLOW_ANONYMOUS_LOGIN=yes bitnami/zookeeper:latest < ~/.sudopassword

[sudo] password for kalinga: 27f4dbfafd0d122a95c9057af9cba8bfdf16171863a0ae2598c0c222cf124de7


## Step 3: Launch the Kafka server instance

In [4]:
!sudo -S docker run -d --name kafka-server \
    --network app-tier \
    -e ALLOW_PLAINTEXT_LISTENER=yes \
    -e KAFKA_CFG_ZOOKEEPER_CONNECT=zookeeper-server:2181 \
    bitnami/kafka:latest < ~/.sudopassword

[sudo] password for kalinga: 91792eec8d1096d05a7cc169771ff0dc2d47ea44057098337566797f6304c89a


## Step 4: Launch event-stream  instance

In [5]:
!sudo -S docker build -t kalinga/event-stream:v0.1 . < ~/.sudopassword

[sudo] password for kalinga: Sending build context to Docker daemon  32.77kB
Step 1/8 : FROM python:3
 ---> b162b5322f1c
Step 2/8 : LABEL Author="Kalinga Bhusan Ray"
 ---> Using cache
 ---> 55e95b0d90db
Step 3/8 : LABEL E-Mail="kalinga-bhusan.ray@tu-ilmenau.de"
 ---> Using cache
 ---> 5e490150ac8b
Step 4/8 : LABEL Desc="This is an image for event stream engiene"
 ---> Using cache
 ---> 430a9dc7ecdd
Step 5/8 : RUN apt-get update &&     apt-get install -y vim
 ---> Using cache
 ---> 63f97c80a8c5
Step 6/8 : ADD process_event_stream.py /
 ---> f32f865fd6c7
Step 7/8 : RUN pip install faust typing_extensions
 ---> Running in ceb63eb8499e
ERROR: Could not find a version that satisfies the requirement faust (from versions: none)
ERROR: No matching distribution found for faust
The command '/bin/sh -c pip install faust typing_extensions' returned a non-zero code: 1


In [6]:
!sudo -S docker run -it -d --name event-stream \
    --network app-tier \
    kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword

[sudo] password for kalinga: f1966e227cf64206c15354231fd02ea8ae6f1cfda89122006cc9dc75ef247b97


In [7]:
!sudo -S docker ps  < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                                    NAMES
f1966e227cf6        kalinga/event-stream:v0.1   "/bin/bash"              14 seconds ago      Up 1 second         80/tcp                                   event-stream
91792eec8d10        bitnami/kafka:latest        "/entrypoint.sh /run…"   3 minutes ago       Up 3 minutes        9092/tcp                                 kafka-server
27f4dbfafd0d        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   3 minutes ago       Up 3 minutes        2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


In [8]:
!ls

Dockerfile	    operations		     README.md	unit-test-1
event-stream.ipynb  process_event_stream.py  test.py


### For existing containers

In [2]:
!sudo -S docker start /zookeeper-server < ~/.sudopassword
!sudo -S docker start /kafka-server < ~/.sudopassword
!sudo -S docker start /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream


In [3]:
!sudo -S docker ps < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                                    NAMES
f1966e227cf6        kalinga/event-stream:v0.1   "/bin/bash"              31 hours ago        Up 15 seconds       80/tcp                                   event-stream
91792eec8d10        bitnami/kafka:latest        "/entrypoint.sh /run…"   31 hours ago        Up 24 seconds       9092/tcp                                 kafka-server
27f4dbfafd0d        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   31 hours ago        Up 28 seconds       2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


#### [In separate terminals]
docker exec -it event-stream bash 
faust -A process_event_stream worker -l info 

docker exec -it kafka-server bash  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties


In [5]:
!sudo -S docker cp event-stream:/process_event_stream.py . < ~/.sudopassword

[sudo] password for kalinga: 

In [9]:
#!sudo -S docker container ls --all < ~/.sudopassword

## Step 5: Unit tests

In [4]:
!sudo -S docker cp unit-test-1 kafka-server:/ < ~/.sudopassword

[sudo] password for kalinga: 

In [ ]:
!sudo -S docker cp unit-test-2 kafka-server:/ < ~/.sudopassword

[kafka-server terminal]  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties < unit-test-1
